## 3. a

Generative Adversarial Network (GAN) adalah jenis arsitektur deep learning yang digunakan untuk menghasilkan data baru yang menyerupai data asli. GAN terdiri dari dua model neural network yang saling berlawanan, yaitu Generator dan Discriminator. Berikut adalah penjelasan cara kerja dari arsitektur GAN berdasarkan gambar yang disertakan:
Komponen GAN
1.	Generator: Model yang bertugas menghasilkan data baru yang menyerupai data asli. Generator mengambil input berupa noise acak (random noise) dan mengubahnya menjadi data yang mirip dengan data pelatihan asli.
2.	Discriminator: Model yang bertugas membedakan antara data asli (dari data pelatihan) dan data palsu (dihasilkan oleh Generator). Discriminator mengeluarkan probabilitas apakah suatu data adalah asli atau palsu.
Proses Kerja GAN
1.	Training Generator:
- Generator mengambil input berupa vektor noise acak dan memprosesnya melalui beberapa lapisan neural network untuk menghasilkan data yang mirip dengan data pelatihan.
- Output dari Generator adalah gambar palsu (fake image) yang mencoba meniru data asli.
2.	Training Discriminator:
- Discriminator diberi dua set data: satu set data asli dari data pelatihan, dan satu set data palsu yang dihasilkan oleh Generator.
- Discriminator bertugas untuk mengklasifikasikan setiap gambar sebagai asli atau palsu.
- Discriminator dilatih untuk memaksimalkan akurasi dalam membedakan antara data asli dan data palsu.
3.	Adversarial Training:
- Proses pelatihan GAN adalah permainan zero-sum antara Generator dan Discriminator. Generator mencoba untuk menipu Discriminator dengan menghasilkan gambar yang semakin mirip dengan data asli, sementara Discriminator berusaha untuk meningkatkan kemampuannya dalam membedakan data asli dan palsu.
- Generator dilatih untuk meminimalkan kemampuan Discriminator dalam mendeteksi bahwa gambar yang dihasilkan adalah palsu.
- Discriminator dilatih untuk memaksimalkan akurasi dalam membedakan gambar asli dan palsu.
Langkah-langkah Pelatihan
1.	Langkah 1: Update Discriminator
- Ambil batch data asli dari data pelatihan dan labelnya sebagai "real".
- Ambil batch data palsu yang dihasilkan oleh Generator dan labelnya sebagai "fake".
- Latih Discriminator dengan data asli dan palsu untuk membedakan keduanya dengan benar.
2.	Langkah 2: Update Generator
- Ambil batch vektor noise acak dan hasilkan data palsu menggunakan Generator.
- Labelkan data palsu ini sebagai "real" untuk melatih Generator.
- Latih Generator untuk meminimalkan kemampuan Discriminator dalam mendeteksi bahwa data palsu adalah palsu.
Tujuan Akhir
Tujuan akhir dari pelatihan GAN adalah mencapai suatu titik di mana Generator menghasilkan data yang sangat mirip dengan data asli sehingga Discriminator tidak dapat membedakan dengan baik antara data asli dan data palsu. Pada titik ini, Generator dianggap telah berhasil mempelajari distribusi data asli dan dapat menghasilkan data yang realistis.
Kesimpulan
GAN menggunakan pendekatan adversarial dengan dua model yang saling berkompetisi untuk menghasilkan data baru yang menyerupai data asli. Generator berusaha untuk menghasilkan data palsu yang realistis, sementara Discriminator berusaha untuk membedakan antara data asli dan palsu. Proses pelatihan yang berkelanjutan ini membantu GAN mencapai hasil yang sangat realistis dalam berbagai aplikasi, seperti generasi gambar, peningkatan resolusi gambar, dan banyak lagi.


## 3. b

generator

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

def build_generator(noise_dim, num_classes, img_size):
    noise_input = layers.Input(shape=(noise_dim,))
    label_input = layers.Input(shape=(num_classes,))
    
    x = layers.Concatenate()([noise_input, label_input])
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dense(img_size * img_size, activation='tanh')(x)  # Use 'tanh' activation for the output layer
    
    img_output = layers.Reshape((img_size, img_size, 1))(x)
    
    model = tf.keras.models.Model([noise_input, label_input], img_output)
    return model


discriminator

In [2]:
def build_discriminator(img_size, num_classes):
    img_input = layers.Input(shape=(img_size, img_size, 1))
    label_input = layers.Input(shape=(num_classes,))
    
    label_embedding = layers.Dense(img_size * img_size)(label_input)
    label_embedding = layers.Reshape((img_size, img_size, 1))(label_embedding)
    
    x = layers.Concatenate()([img_input, label_embedding])
    
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.models.Model([img_input, label_input], x)
    return model


train the GAN

In [3]:
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Hyperparameters
noise_dim = 100
num_classes = 10
img_size = 28
batch_size = 64
epochs = 50  # Reduced number of epochs

# Load and preprocess data
(x_train, y_train), (_, _) = mnist.load_data()
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)
y_train = to_categorical(y_train, num_classes)

# Build and compile the discriminator
discriminator = build_discriminator(img_size, num_classes)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Build the generator
generator = build_generator(noise_dim, num_classes, img_size)

# Create GAN model
noise_input = layers.Input(shape=(noise_dim,))
label_input = layers.Input(shape=(num_classes,))
img_output = generator([noise_input, label_input])

discriminator.trainable = False
validity = discriminator([img_output, label_input])

gan = tf.keras.models.Model([noise_input, label_input], validity)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

# Training loop
for epoch in range(epochs):
    # Train discriminator
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs, labels = x_train[idx], y_train[idx]

    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    gen_imgs = generator.predict([noise, labels])

    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    d_loss_real = discriminator.train_on_batch([imgs, labels], real)
    d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train generator
    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    valid_y = np.ones((batch_size, 1))

    g_loss = gan.train_on_batch([noise, labels], valid_y)

    # Print the progress
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, D loss: {d_loss[0]}, G loss: {g_loss}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step  


C:\Users\asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\backend\tensorflow\trainer.py:71: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0/50, D loss: 0.8981834650039673, G loss: [array(0.8197177, dtype=float32), array(0.8197177, dtype=float32), array(0.4921875, dtype=float32)]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Epoch 10/50, D loss: 2.183459758758545, G loss: [array(2.2586024, dtype=float32), array(2.2586024, dtype=float32), array(0.04758523, dtype=float32)]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━

FID Score

In [4]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from scipy.linalg import sqrtm

def calculate_fid(generator, real_images, noise_dim, num_classes, num_images=1000):
    # Load InceptionV3 model
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

    # Generate fake images
    noise = np.random.normal(0, 1, (num_images, noise_dim))
    labels = np.random.randint(0, num_classes, num_images)
    labels = to_categorical(labels, num_classes)
    fake_images = generator.predict([noise, labels])
    fake_images = np.repeat(fake_images, 3, axis=-1)  # Convert to 3 channels
    fake_images = preprocess_input(tf.image.resize(fake_images, (299, 299)))

    # Resize real images to match InceptionV3 input size
    real_images = np.repeat(real_images, 3, axis=-1)  # Convert to 3 channels
    real_images_resized = preprocess_input(tf.image.resize(real_images, (299, 299)))

    # Calculate activations
    act1 = model.predict(real_images_resized)
    act2 = model.predict(fake_images)

    # Calculate mean and covariance
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

    # Calculate FID
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

# Calculate FID score
fid_score = calculate_fid(generator, x_train[:1000], noise_dim, num_classes)
print(f"FID Score: {fid_score}")


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 48s 1us/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step
FID Score: 111.08509142725205
